In [ ]:
#https://github.com/anujshah1003/Tensorboard-own-image-data-image-features-embedding-visualization

In [3]:
# -*- coding: utf-8 -*-


import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.__version__

PATH = os.getcwd()

LOG_DIR = PATH+ '/embedding-logs'
#metadata = os.path.join(LOG_DIR, 'metadata2.tsv')

In [4]:
PATH = 'image_dataset'
#%%
data_path = PATH 
data_dir_list = os.listdir(data_path)

img_data=[]
img_list=os.listdir(data_path)



In [71]:
img_list = [img for img in img_list if img != '.ipynb_checkpoints']

In [76]:
img_data = []
for img in img_list:
    input_img=cv2.imread(PATH + '/'+ img )
    input_img_resize=cv2.resize(input_img, (224,224))
    img_data.append(input_img_resize)
    
                
img_data = np.array(img_data)

In [74]:
import csv
import tensorflow as tf
import tensorflow_hub as hub

In [75]:
img_data = img_data[:1000]

In [8]:
img_data.shape

(1000, 224, 224, 3)

In [9]:
tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.FATAL)

module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/3",
                    trainable=False)#, tags={"train"})
features = module(inputs=dict(images=img_data, batch_norm_momentum=0.997),
                  signature="image_feature_vector_with_bn_hparams")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    result = sess.run(features)

In [10]:
features = tf.Variable(result, name='features')

In [11]:
features.shape

TensorShape([Dimension(1000), Dimension(2048)])

In [15]:
%store -r full_dataset

In [16]:
del full_dataset

In [77]:
import pandas as pd
accidents = pd.read_csv('accidents_report.csv')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (115,116) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [92]:
accidents.columns.tolist()

['Unnamed: 0',
 'Unit_ID',
 'Crime_No',
 'Accident_DateTime',
 'Fatal_NonFatal',
 'Accident_Classification',
 'Severity',
 'Collision_Type',
 'Accident_Spot',
 'Accident_Location',
 'Accident_Zone',
 'NoOfVehicle_Involved',
 'Junction_Control',
 'Road_Character',
 'Road_Type',
 'Road_Classification',
 'Road_Width',
 'Shoulder_Width',
 'Seperation',
 'Surface_Type',
 'Surface_Condition',
 'Road_Condition',
 'weather',
 'Location_Type',
 'Main_Cause',
 'Hit_Run',
 'XY_Map',
 'X_Value',
 'AccidentLocation_Map',
 'Collision_Diagram',
 'Y_Value',
 'Route',
 'KM',
 'HundredM',
 'Node_Map',
 'Node1',
 'Node2',
 'Accident_City',
 'Accident_Road',
 'Reporting_Officer',
 'LandMark_First',
 'LandMark_Second',
 'Distance_LandMark_First',
 'Distance_LandMark_Second',
 'Junction_Second_Name',
 'Distance_Second_Road',
 'Research_Code',
 'Accident_Description',
 'Action_Special_Note',
 'UserID',
 'Inspection_Officer_Name',
 'Inspection_Officer_Rank',
 'Inspection_Officer_Code',
 'Actual_DateOf_Occuran

In [93]:
accidents['Road_Type_val'].value_counts()

NH                     58738
State Highway          38605
City or Town Road      32347
Village Road           27362
Others                 22709
Two way                 6253
Not Applicable          3352
One way                 2533
Service Road            2011
Major District Road     1836
Residential Street      1589
Minor District Road     1310
Arterial                 824
Expressway               633
Mixed                    466
Sub Arterial             293
Forest Road              238
Feeder Road              101
Name: Road_Type_val, dtype: int64

In [102]:
severities = []
road_type = []
for img in img_list[:1000]:
    crime_no = img[:10]
    unit_id = img[10:-4]    
    if unit_id == '':
        severities.append('JUNK')
        road_type.append('JUNK')
        continue
    res = accidents.query('Crime_No=={0} and Unit_ID=={1}'.format(crime_no,unit_id))
    if res.shape[0] == 0:
        severities.append('JUNK')
        road_type.append('JUNK')
        continue
    severities.append(res['Severity_val'].iloc[0])
    road_type.append(res['Road_Type_val'].iloc[0])
#    break

In [101]:
len(road_type)

932

In [103]:
#with open(metadata, 'w') as metadata_file:
#    for row in range(210):
#        c = y[row]
#        metadata_file.write('{}\n'.format(c))
metadata_file = open(os.path.join(LOG_DIR, 'metadata_4_classes.tsv'), 'w')
metadata_file.write('Class\tSeverity\tRoadType\n')
k=100 # num of samples in each class
j=0
#for i in range(210):
#    metadata_file.write('%06d\t%s\n' % (i, names[y[i]]))
for i in range(len(severities)):
    #metadata_file.write('{}\n'.format(severities[i]))
    if i%k==0:
        j=j+1
    metadata_file.write('{0}\t{1}\t{2}\n'.format(j, 
                                                 severities[i], 
                                                 road_type[i]))
    
metadata_file.close()


In [104]:
set(severities), set(road_type)

({'Damage Only',
  'Fatal',
  'Grievous Injury',
  'JUNK',
  'Not Applicable',
  'Simple Injury'},
 {'Arterial',
  'City or Town Road',
  'Expressway',
  'Feeder Road',
  'Forest Road',
  'JUNK',
  'Mixed',
  'NH',
  'Not Applicable',
  'One way',
  'Others',
  'Residential Street',
  'Service Road',
  'State Highway',
  'Sub Arterial',
  'Two way',
  'Village Road'})

In [12]:
    
# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding
    Args:
      data: NxHxW[x3] tensor containing the images.
    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data

In [13]:
#%%
sprite = images_to_sprite(img_data)
cv2.imwrite(os.path.join(LOG_DIR, 'sprite_4_classes.png'), sprite)
#scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)


True

In [67]:

#%%
with tf.Session() as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images_4_classes.ckpt'))
    
    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata_4_classes.tsv')
    # Comment out if you don't want sprites
    embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite_4_classes.png')
    embedding.sprite.single_image_dim.extend([img_data.shape[1], img_data.shape[1]])
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

In [110]:
#accident description

In [112]:
#!pip install langdetect

In [114]:
from langdetect import detect

In [115]:
#detect("Ein, zwei, drei, vier")

In [116]:
accidents['Accident_Description'][100:150]

100    ಅಪಘಾತ ನಡೆದ ಸ್ಥಳವು ಪಾಣೆಮಂಗಳೂರು ಗ್ರಾಮದ ಪಾಣೆಮಂಗಳೂ...
101    N.H.66 IN FRONT OF BHUVANESHWARAI STORES SALIG...
102                               Near CoffeDay B.B.Road
103                                                    .
104    ಅಪಘಾತ ನಡೆದ ಸ್ಥಳದ ಚಕ್ಕುಬಂದಿ  ಪೂರ್ವಕ್ಕೆ  ಸಿಂಗಸಂದ...
105    ರಾಯಚೂರ ಬೆಳಗಾವಿ ರಾಜ್ಯ ಹೆದ್ದಾರಿ ಮೆಲೆ ತುಂಬ ಗ್ರಾಮದ...
106    ಸದರಿ ಪ್ರಕರಣದ ಅಪಘಾತವು ಚಿಕ್ಕಜಾಜೂರು ಠಾಣಾ ವ್ಯಾಪ್ತಿ...
107    ಶಿವಮೊಗ್ಗ ಪಶ್ವಿಮ ಸಂಚಾರಿ ಪೊಲೀಸ್ ಠಾಣಾ ಗುನ್ನೆ ನಂ:1...
108    ಬೆಂಗಳೂರು ಜಿಲ್ಲೆ, ಆನೇಕಲ್ ತಾಲ್ಲೊಕು, ಅತ್ತಿಬೆಲೆ ಹೋ...
109    MARAKUMBI VILLAGE LIMITS NEAR MARAKUMBI BRIDGE...
110    ಚಾಮರಾಜನಗರ ಗ್ರಾಮಾಂತರ ಪೊಲೀಸ್ ಠಾಣಾ ಸರಹದ್ದು ಚಾ,ನಗರ...
111    HALAKI VILALGE LIMITS NEAR HALAKI VILLAGE ON Y...
112                                Tidagundi Bridge Near
113    BHUDIGOPPA VILLAGE LIMITS NEAR GOVT JUNGALE, O...
114                                       ಕೃತ್ಯಸ್ಥಳದಲ್ಲಿ
115    ದಾವಣಗೆರೆ ಉತ್ತರ ಸಂಚಾರ ಪೊಲೀಸ್ ಠಾಣಾ ಸರಹದ್ದಿನ ಎ.ಪಿ...
116    ಅಪಫಾತ ಸ್ಥಳವು ಬಂಟ್ವಾಳ ತಾಲೂಕು ತುಂಬೆ  ಗ್ರಾಮದ ತುಂಬ...
117    NEAR KRISHNAPURA NURSERY

In [132]:
accidents_with_desc = accidents[accidents.apply(lambda x: len(str(x['Accident_Description']).split()) > 1, axis=1)]

In [144]:
from inltk.inltk import get_embedding_vectors, get_sentence_encoding
from inltk.inltk import setup

In [149]:
#setup('kn')
#setup('en')

In [153]:
accident_desc_sent_emb = []
#for rec in accidents_with_desc['Accident_Description']:
    #detect(rec) == 'kn'
for rec in accidents_with_desc.iterrows():
    sent = rec[1]['Accident_Description']
    print(sent)
    print(".....")
    if  detect(sent) == 'kn':
        sent_emb = get_sentence_encoding(sent, 'kn')
    else:
        sent_emb = get_sentence_encoding(sent, 'en')
    accident_desc_sent_emb.append(sent_emb)

BELAVI LIMITS ON HUKKERI TO SHIRAHATTI B.K ROAD
.....


Exception: Language code should be one of ['bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'ne', 'or', 'pa', 'sa', 'ta', 'ur'] and not en